In [ ]:
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "8"   # np. 8 rdzeni

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import datacompy
import os, sys
import numpy as np
import re
import ast
from pathlib import Path
import openpyxl
import itertools
import torch
import gc
import parquet

# narzedzia
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    cross_val_predict,
    learning_curve,
    RepeatedStratifiedKFold,
    GridSearchCV,
    RandomizedSearchCV
)
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score, accuracy_score, pairwise_distances, make_scorer, precision_score, f1_score, roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from scipy import stats
from scipy.stats import chi2_contingency, f_oneway, friedmanchisquare, wilcoxon
from scipy.spatial import distance
from joblib import dump, load

# modele
from xgboost import XGBClassifier, XGBRFClassifier
#from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# methods
from imblearn.under_sampling import ClusterCentroids, NearMiss
from scipy.optimize import differential_evolution
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, HDBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor, KNeighborsClassifier, NearestNeighbors
from sklearn.inspection import permutation_importance
from scipy.spatial.distance import euclidean
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from ctgan import CTGAN

In [88]:
df_unsw = pd.read_csv('D:\\ml\\undersampling_data\\data\\cicunsw\\CICFlowMeter.csv')

In [91]:
df_unsw["Label"] = df_unsw["Label"].apply(lambda x: 0 if x == 'Benign' else 1)

In [70]:
df_unsw.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,175.45.176.2-149.171.126.16-23357-80-6,175.45.176.2,23357,149.171.126.16,80,6,22/01/2015 07:50:15 AM,214392,9,21,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,175.45.176.0-149.171.126.16-13284-80-6,175.45.176.0,13284,149.171.126.16,80,6,22/01/2015 07:50:13 AM,2376792,9,3,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,175.45.176.2-149.171.126.16-13792-5555-6,175.45.176.2,13792,149.171.126.16,5555,6,22/01/2015 07:50:16 AM,131350,10,3,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,175.45.176.0-149.171.126.15-39500-80-6,175.45.176.0,39500,149.171.126.15,80,6,22/01/2015 07:50:18 AM,164796,6,3,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,175.45.176.0-149.171.126.14-29309-3000-6,175.45.176.0,29309,149.171.126.14,3000,6,22/01/2015 07:50:19 AM,163418,6,3,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [71]:
df_unsw.describe()

,Src Port,Dst Port,Protocol,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
count,3.540241e+06,3.540241e+06,3.540241e+06,3.540241e+06,3.540241e+06,3.540241e+06,3.540241e+06,3.540241e+06,3.540241e+06,3.540241e+06,...,3.540241e+06,3.540241e+06,3.540241e+06,3.540241e+06,3.540241e+06,3.540241e+06,3.540241e+06,3.540241e+06,3.540241e+06,3540241.0
mean,3.187573e+04,1.641632e+04,7.759312e+00,3.487602e+05,2.266156e+01,2.938711e+01,1.878354e+03,2.342127e+04,1.500797e+02,1.094084e+01,...,2.759795e+01,9.482037e+03,2.792051e+03,1.286217e+04,7.603646e+03,4.695143e+04,1.825652e+03,4.866172e+04,4.552857e+04,1.0
std,1.948902e+04,2.038958e+04,4.032029e+00,2.748282e+06,6.852967e+01,1.010743e+02,5.530137e+04,1.282813e+05,3.216338e+02,3.996782e+01,...,8.915630e+00,3.810417e+05,2.550465e+05,5.837946e+05,3.273663e+05,1.289154e+06,1.115097e+05,1.309870e+06,1.278377e+06,0.0
min,1.000000e+01,1.000000e+01,6.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,8.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0
25%,1.472900e+04,8.000000e+01,6.000000e+00,3.430000e+02,1.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0
50%,3.178900e+04,6.434000e+03,6.000000e+00,1.229000e+03,2.000000e+00,2.000000e+00,9.000000e+01,1.220000e+02,4.500000e+01,0.000000e+00,...,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0
75%,4.884600e+04,3.090600e+04,6.000000e+00,5.235100e+04,2.200000e+01,2.300000e+01,4.740000e+02,4.986000e+03,1.140000e+02,0.000000e+00,...,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0
max,6.553500e+04,6.553500e+04,1.700000e+01,1.199995e+08,2.003800e+04,1.102100e+04,2.605270e+07,1.421599e+07,1.476000e+03,1.472000e+03,...,4.000000e+01,1.109349e+08,7.417760e+07,1.109349e+08,1.109349e+08,1.198828e+08,7.214360e+07,1.198828e+08,1.198828e+08,1.0


In [72]:
df_unsw["Protocol"].value_counts()

Protocol
6     2974024
17     566217
Name: count, dtype: int64

In [92]:
df_unsw['Dst Port System'] = (df_unsw['Dst Port'] < 1024).astype(int)
df_unsw['Dst Port User'] = ((df_unsw['Dst Port'] >= 1024) & (df_unsw['Dst Port'] < 49152)).astype(int)
df_unsw['Dst Port Dynamic'] = (df_unsw['Dst Port'] >= 49152).astype(int)

col = df_unsw.pop('Dst Port System')
col2 = df_unsw.pop('Dst Port User')
col3 = df_unsw.pop('Dst Port Dynamic')
dst_col = df_unsw.columns.get_loc('Protocol')
df_unsw.insert(dst_col, 'Dst Port System', col)
df_unsw.insert(dst_col + 1, 'Dst Port User', col2)
df_unsw.insert(dst_col + 2, 'Dst Port Dynamic', col3)

df_unsw = df_unsw.drop(columns=['Dst Port'])

In [93]:
df_unsw = df_unsw.drop(columns=['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Timestamp'])

In [94]:
df_unsw.rename(columns={'Protocol': 'Protocol TCP'}, inplace=True)
df_unsw["Protocol TCP"] = df_unsw["Protocol TCP"].apply(lambda x: 1 if x == 6 else 0)

In [95]:
columns = ['Flow Duration', 'Total Fwd Packet', 'Total Bwd packets', 'Total Length of Fwd Packet', 'Total Length of Bwd Packet', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg','Bwd Bytes/Bulk Avg', 'Bwd Packet/Bulk Avg', 'Bwd Bulk Rate Avg', 'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'FWD Init Win Bytes', 'Bwd Init Win Bytes', 'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min']

scaler = StandardScaler()
df_unsw[columns] = scaler.fit_transform(df_unsw[columns])

with open('D:\\ml\\undersampling_data\\models\\scalar_cicunsw.pkl', 'wb') as f:
    pickle.dump(scaler,f)

In [77]:
df_unsw.head()

,Dst Port System,Dst Port User,Dst Port Dynamic,Protocol TCP,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,1,0,0,1,-0.048892,-0.199353,-0.082980,-0.026950,0.008908,0.136554,...,-0.852206,-0.024885,-0.010947,-0.022032,-0.023227,-0.03642,-0.016372,-0.03715,-0.035614,1
1,1,0,0,1,0.737927,-0.199353,-0.261067,-0.020368,-0.182577,0.117899,...,-0.852206,-0.024885,-0.010947,-0.022032,-0.023227,-0.03642,-0.016372,-0.03715,-0.035614,1
2,0,1,0,1,-0.079108,-0.184760,-0.261067,0.102812,-0.182577,4.072708,...,-0.852206,-0.024885,-0.010947,-0.022032,-0.023227,-0.03642,-0.016372,-0.03715,-0.035614,1
3,1,0,0,1,-0.066938,-0.243129,-0.261067,-0.020042,-0.182577,0.730397,...,-0.852206,-0.024885,-0.010947,-0.022032,-0.023227,-0.03642,-0.016372,-0.03715,-0.035614,1
4,0,1,0,1,-0.067439,-0.243129,-0.261067,-0.026733,-0.182577,0.155208,...,-0.852206,-0.024885,-0.010947,-0.022032,-0.023227,-0.03642,-0.016372,-0.03715,-0.035614,1


In [78]:
df_unsw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3540241 entries, 0 to 3540240
Data columns (total 81 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Dst Port System             int64  
 1   Dst Port User               int64  
 2   Dst Port Dynamic            int64  
 3   Protocol TCP                int64  
 4   Flow Duration               float64
 5   Total Fwd Packet            float64
 6   Total Bwd packets           float64
 7   Total Length of Fwd Packet  float64
 8   Total Length of Bwd Packet  float64
 9   Fwd Packet Length Max       float64
 10  Fwd Packet Length Min       float64
 11  Fwd Packet Length Mean      float64
 12  Fwd Packet Length Std       float64
 13  Bwd Packet Length Max       float64
 14  Bwd Packet Length Min       float64
 15  Bwd Packet Length Mean      float64
 16  Bwd Packet Length Std       float64
 17  Flow Bytes/s                float64
 18  Flow Packets/s              float64
 19  Flow IAT Mean        

In [96]:
df_unsw = df_unsw.apply(pd.to_numeric, downcast='integer')
df_unsw = df_unsw.apply(pd.to_numeric, downcast='float')

In [80]:
df_unsw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3540241 entries, 0 to 3540240
Data columns (total 81 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Dst Port System             int8   
 1   Dst Port User               int8   
 2   Dst Port Dynamic            int8   
 3   Protocol TCP                int8   
 4   Flow Duration               float32
 5   Total Fwd Packet            float32
 6   Total Bwd packets           float32
 7   Total Length of Fwd Packet  float32
 8   Total Length of Bwd Packet  float32
 9   Fwd Packet Length Max       float32
 10  Fwd Packet Length Min       float32
 11  Fwd Packet Length Mean      float32
 12  Fwd Packet Length Std       float32
 13  Bwd Packet Length Max       float32
 14  Bwd Packet Length Min       float32
 15  Bwd Packet Length Mean      float32
 16  Bwd Packet Length Std       float32
 17  Flow Bytes/s                float32
 18  Flow Packets/s              float32
 19  Flow IAT Mean        

In [97]:
df_unsw.to_csv('D:\\ml\\undersampling_data\\data\\cicunsw\\CICFlowMeter_processed.csv', index=False)

In [82]:
df_unsw["Label"].value_counts()

Label
1    3540241
Name: count, dtype: int64

In [98]:
X = df_unsw.drop(columns=['Label'])
y = df_unsw['Label']
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,        # 20% danych do testu
    random_state=42,      # dla powtarzalności
    stratify=y            # zachowaj proporcje klas
)

X_train.shape, X_test.shape

((2478168, 80), (1062073, 80))

In [99]:
df_unsw.to_parquet('D:\\ml\\undersampling_data\\data\\cicunsw\\CICUNSW_processed.parquet', compression='snappy', index=False)

In [100]:
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

train_df.to_parquet('D:\\ml\\undersampling_data\\data\\cicunsw\\CICUNSW_train.parquet', compression='snappy', index=False)
test_df.to_parquet('D:\\ml\\undersampling_data\\data\\cicunsw\\CICUNSW_test.parquet', compression='snappy', index=False)

In [101]:
y_test.value_counts()

Label
0    1035198
1      26875
Name: count, dtype: int64

### Oversampling

In [10]:
train_df = pd.read_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\CICUNSW_train.parquet")
y_train = train_df["Label"]
X_train = train_df.drop(columns=["Label"])

In [11]:
file_path1 = "D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\smote_data.csv"
file_path2 = "D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\smote3_data.csv"
#zamiana jesli istnieje to wczytaj plik jesli nie to stworz
coun = Counter(y_train)
majority_class = max(coun, key=coun.get)
minority_class = min(coun, key=coun.get)
missing_samples = coun[majority_class] - coun[minority_class]
print("Before", coun)
smote = SMOTE()
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

smote2 = SMOTE(sampling_strategy={minority_class: coun[minority_class] + 2 * missing_samples}, random_state=42)
X_train_sm3, y_train_sm3 = smote2.fit_resample(X_train, y_train)

train_data_smote = pd.concat([X_train_sm, y_train_sm], axis=1)          #polaczenie danych wygenerowanych X_train oraz y_train
train_data_smote3 = pd.concat([X_train_sm3, y_train_sm3], axis=1)

#smote generated data
train_data_smote['generated_by_smote'] = ['original' if i < len(train_df) else 'smote' for i in range(len(train_data_smote))]
smote_data = train_data_smote[train_data_smote['generated_by_smote'] == 'smote'].drop('generated_by_smote', axis=1)
smote_data["source"] = "smote"
if not os.path.exists(file_path1):
    smote_data.to_csv(file_path1, index=False)
else:
    print(f'Plik istnieje pod ścieżką: {file_path1}')

#smote3 generated data
train_data_smote3['generated_by_smote'] = ['original' if i < len(train_df) else 'smote' for i in range(len(train_data_smote3))]
smote_data3 = train_data_smote3[train_data_smote3['generated_by_smote'] == 'smote'].drop('generated_by_smote', axis=1)
smote_data3["source"] = "smote"
if not os.path.exists(file_path2):
    smote_data3.to_csv(file_path2, index=False)
else:
    print(f'Plik istnieje pod ścieżką: {file_path2}')

con1 = Counter(y_train_sm)
print("After", con1)
con2 = Counter(y_train_sm3)
print("After generation 3x SMOTE", con2)
#pd.Series(y_train_sm).value_counts().plot.bar()
#pd.Series(y_train_sm3).value_counts().plot.bar()

Before Counter({0: 2415460, 1: 62708})


c:\Users\Mati\AppData\Local\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] Nie można odnaleźć określonego pliku
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\Mati\AppData\Local\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "c:\Users\Mati\AppData\Local\anaconda3\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Mati\AppData\Local\anaconda3\Lib\subprocess.py", line 1039, in __init__
    self._execute_child(args, executable, preexec_fn, close

After Counter({0: 2415460, 1: 2415460})
After generation 3x SMOTE Counter({1: 4768212, 0: 2415460})


In [16]:
smote_data['label'].value_counts()

label
1    2352752
Name: count, dtype: int64

In [103]:
smote_data.to_parquet('D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\smote_data.parquet', compression='snappy', index=False)
smote_data3.to_parquet('D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\smote3_data.parquet', compression='snappy', index=False)

In [17]:
file_path2 = "D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\borderline_data.csv"
file_path1 = "D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\borderline3_data.csv"
#generate new data by borderLineSMOTE
coun3 = Counter(y_train)
majority_class = max(coun3, key=coun3.get)
minority_class = min(coun3, key=coun3.get)
missing_samples = coun3[majority_class] - coun3[minority_class]
print("Before", coun3)

brdsmote = BorderlineSMOTE(random_state=42)
X_train_bsm, y_train_bsm = brdsmote.fit_resample(X_train, y_train)

brdsmote3 = BorderlineSMOTE(sampling_strategy={minority_class: coun3[minority_class] + 2 * missing_samples}, random_state=42)
X_train_bsm3, y_train_bsm3 = brdsmote3.fit_resample(X_train, y_train)

con4 = Counter(y_train_bsm)
print("After", con4)
con5 = Counter(y_train_bsm3)
print("After generated 3x BorderlineSMOTE", con5)

train_data_borderline_smote = pd.concat([X_train_bsm, y_train_bsm], axis=1)          #polaczenie danych wygenerowanych X_train oraz y_train
train_data_borderline_smote3 = pd.concat([X_train_bsm3, y_train_bsm3], axis=1)

#borderline smote generated data
train_data_borderline_smote['generated_by_borderline_smote'] = ['original' if i < len(train_df) else 'brd smote' for i in range(len(train_data_borderline_smote))]
boarderline_smote_data = train_data_borderline_smote[train_data_borderline_smote['generated_by_borderline_smote'] == 'brd smote'].drop('generated_by_borderline_smote', axis=1)
boarderline_smote_data["source"]="borderline smote"
boarderline_smote_data = boarderline_smote_data[boarderline_smote_data['source'] != 'original']
if not os.path.exists(file_path2):
    boarderline_smote_data.to_csv(file_path2, index=False)
else:
    print(f'Plik istnieje pod ścieżką: {file_path2}')
    
#borderline smote3 generated data
train_data_borderline_smote3['generated_by_borderline_smote'] = ['original' if i < len(train_df) else 'brd smote' for i in range(len(train_data_borderline_smote3))]
boarderline_smote_data3 = train_data_borderline_smote3[train_data_borderline_smote3['generated_by_borderline_smote'] == 'brd smote'].drop('generated_by_borderline_smote', axis=1)
boarderline_smote_data3["source"]="borderline smote"
boarderline_smote_data3 = boarderline_smote_data3[boarderline_smote_data3['source'] != 'original']
if not os.path.exists(file_path1):
    boarderline_smote_data3.to_csv(file_path1, index=False)
else:
    print(f'Plik istnieje pod ścieżką: {file_path1}')


Before Counter({0: 2415460, 1: 62708})
After Counter({0: 2415460, 1: 2415460})
After generated 3x BorderlineSMOTE Counter({1: 4768212, 0: 2415460})


In [20]:
boarderline_smote_data3["Label"].value_counts()

Label
1    4705504
Name: count, dtype: int64

In [18]:
boarderline_smote_data.to_parquet('D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\borderline_data.parquet', compression='snappy', index=False)
boarderline_smote_data3.to_parquet('D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\borderline3_data.parquet', compression='snappy', index=False)
smote_data.to_parquet('D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\smote_data.parquet', compression='snappy', index=False)
smote_data3.to_parquet('D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\smote3_data.parquet', compression='snappy', index=False)

In [109]:
file_path3 = "D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\GAN_data.csv"        #sciezka wraz z nazwa pod jaka wygenerowac plik
file_path4 = "D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\GAN3_data.csv"        #sciezka wraz z nazwa pod jaka wygenerowac plik

# Chunking parameters
chunk_rows = 30000      # ile wierszy mniejszości na jedną porcję (dostosuj do VRAM/RAM)
max_shards = None       # np. 10 by ograniczyć liczbę porcji; None = bez limitu

# CTGAN
CTGAN_KW = dict(
    epochs=10,
    batch_size=16,
    generator_dim=(64, 64),
    discriminator_dim=(64, 64),
    embedding_dim=16,
    pac=1,
    cuda=True,          # będzie zbijane na False przy OOM
)

# ===== pomocnicze =====
def detect_discrete_columns(df, label_col="Label", max_int_card=50):
    """Zbuduj listę kolumn dyskretnych dla CTGAN (stare API oczekuje takiej listy)."""
    disc = []
    for c in df.columns:
        if c == label_col:
            disc.append(c); continue
        dt = df[c].dtype
        if str(dt) in ("object", "bool") or "category" in str(dt):
            disc.append(c)
        elif np.issubdtype(dt, np.integer) and df[c].nunique(dropna=True) <= max_int_card:
            disc.append(c)
    return sorted(list(set(disc)))

def make_ctgan():
    try:
        from ctgan import CTGAN
    except Exception as e:
        raise RuntimeError("Brak pakietu ctgan. Zainstaluj: pip install ctgan") from e
    return CTGAN(**CTGAN_KW)

def fit_with_fallback(ctgan, data, discrete_columns):
    """Spróbuj na GPU, w razie OOM spadnij na CPU i oczyść pamięć GPU."""
    try:
        ctgan.fit(data, discrete_columns)
        return ctgan
    except RuntimeError as e:
        msg = str(e).lower()
        if "cuda" in msg or "out of memory" in msg or "cublas" in msg:
            # fallback na CPU
            try:
                import torch
                torch.cuda.empty_cache()
            except Exception:
                pass
            # przeinstancjuj CTGAN na CPU
            cpu_kwargs = {**CTGAN_KW, "cuda": False}
            from ctgan import CTGAN
            ctgan_cpu = CTGAN(**cpu_kwargs)
            ctgan_cpu.fit(data, discrete_columns)
            return ctgan_cpu
        raise

def generate_from_shards(minority_df, deficit_total, discrete_columns,
                         multiplier=1, chunk_rows=30000, max_shards=None):
    """
    minority_df: tylko mniejszość (Label == minority_label)
    deficit_total: ile łącznie przykładów musimy dorobić, by zbalansować.
    multiplier: 1 dla 'GAN' (wyrównanie), 2 dla 'GAN3' (×2 mniejszość).
    """
    target_total = int(deficit_total * multiplier)
    if target_total <= 0:
        return pd.DataFrame(columns=minority_df.columns)

    # Pokrój minority na porcje
    n = len(minority_df)
    if chunk_rows <= 0:
        chunk_rows = n  # jedna porcja
    shard_indices = list(range(0, n, chunk_rows))
    if max_shards is not None:
        shard_indices = shard_indices[:max_shards]

    samples_left = target_total
    generated_parts = []

    for i, start in enumerate(shard_indices, 1):
        end = min(start + chunk_rows, n)
        shard = minority_df.iloc[start:end].copy()

        # Rozdziel ile generujemy z tej porcji (proporcjonalnie do rozmiaru porcji)
        # + nigdy nie przekraczaj tego co zostało
        share = (len(shard) / n)
        to_make = int(round(target_total * share))
        to_make = min(to_make, samples_left)
        if to_make <= 0:
            continue

        # Każda porcja: nowy, mały model (czyści pamięć po zakończeniu)
        ctgan = make_ctgan()
        ctgan = fit_with_fallback(ctgan, shard, discrete_columns)

        synth = ctgan.sample(to_make)
        generated_parts.append(synth)

        samples_left -= to_make

        # porządkowanie pamięci
        del ctgan; gc.collect()
        try:
            import torch
            torch.cuda.empty_cache()
        except Exception:
            pass

        if samples_left <= 0:
            break

    if samples_left > 0:
        # Jeżeli zaokrąglenia zostawiły nam ogon — dorób z ostatniej porcji
        last = minority_df.iloc[shard_indices[-1]:min(shard_indices[-1]+chunk_rows, n)].copy()
        ctgan = make_ctgan()
        ctgan = fit_with_fallback(ctgan, last, discrete_columns)
        synth_tail = ctgan.sample(samples_left)
        generated_parts.append(synth_tail)
        del ctgan; gc.collect()
        try:
            import torch
            torch.cuda.empty_cache()
        except Exception:
            pass

    if not generated_parts:
        return pd.DataFrame(columns=minority_df.columns)

    out = pd.concat(generated_parts, ignore_index=True)
    return out

# ===== GŁÓWNY PRZEPŁYW =====

# Zakładam, że masz df w pamięci:
# df = df.drop(columns=["Unnamed: 0"], errors="ignore")
df_gan = train_df

label_col = "Label"
counts = df_gan[label_col].value_counts().sort_index()
if set(counts.index) != {0, 1}:
    raise ValueError(f"Oczekiwałem klas 0/1 w kolumnie '{label_col}', mam: {counts.index.tolist()}")

majority_label = counts.idxmax()
minority_label = 1 - majority_label
deficit = counts[majority_label] - counts[minority_label]

# Kolumny dyskretne do CTGAN
discrete_columns = detect_discrete_columns(df_gan, label_col=label_col, max_int_card=50)

# Dane mniejszości
minority_df = df_gan[df_gan[label_col] == minority_label].reset_index(drop=True)

# --- Wersja GAN (wyrównanie 1:1) ---
df_GAN = generate_from_shards(
    minority_df=minority_df,
    deficit_total=deficit,
    discrete_columns=discrete_columns,
    multiplier=1,
    chunk_rows=chunk_rows,
    max_shards=max_shards
)

balanced_data = pd.concat([df_gan, df_GAN], ignore_index=True)

# --- Wersja GAN3 (×2 mniejszość) ---
df_GAN3 = generate_from_shards(
    minority_df=minority_df,
    deficit_total=deficit,
    discrete_columns=discrete_columns,
    multiplier=2,
    chunk_rows=chunk_rows,
    max_shards=max_shards
)

balanced_data3 = pd.concat([df_gan, df_GAN3], ignore_index=True)

# Raporty
y_train = df_gan[label_col]                    # Twoje "Before" z oryginału
y_train_gan  = balanced_data[label_col]
y_train_gan3 = balanced_data3[label_col]

print("Before:", Counter(y_train))
print("After (GAN):", Counter(y_train_gan))
print("After (GAN3):", Counter(y_train_gan3))

# Oznacz generatywne
df_GAN["source"]  = "gan"
df_GAN3["source"] = "gan"

# Zapis
if not os.path.exists(file_path3):
    df_GAN.to_csv(file_path3, index=False)
else:
    print(f"Plik istnieje pod ścieżką: {file_path3}")

if not os.path.exists(file_path4):
    df_GAN3.to_csv(file_path4, index=False)
else:
    print(f"Plik istnieje pod ścieżką: {file_path4}")


Before: Counter({0: 2415460, 1: 62708})
After (GAN): Counter({0: 2415460, 1: 2415460})
After (GAN3): Counter({1: 4768212, 0: 2415460})


In [110]:
df_GAN.to_parquet('D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\GAN_data.parquet', compression='snappy', index=False)
df_GAN3.to_parquet('D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\GAN3_data.parquet', compression='snappy', index=False)